In [3]:
import nltk
import json
import numpy as np
import pickle

In [4]:
with open("testing_data.pickle", "rb") as input_file:
    testing_data = pickle.load(input_file)

In [4]:
emb_mat = np.load("word_embedding_matrix.npy").astype(np.float)

with open("vocabulary.pickle", "rb") as input_file:
    voc = pickle.load(input_file)

In [4]:
with open("training_data.pickle", "rb") as input_file:
    training_data = pickle.load(input_file)

In [22]:
np.random.shuffle(training_data)

In [23]:
print(training_data[0])

{'question': ['how', 'many', 'people', 'watched', 'the', 'apollo', '11', 'landing', '?'], 'context': ['the', 'landing', 'of', 'apollo', '11', 'was', 'an', 'event', 'watched', 'by', 'over', '500', 'million', 'people', 'around', 'the', 'world', 'and', 'is', 'widely', 'recognized', 'as', 'one', 'of', 'the', 'defining', 'moments', 'of', 'the', '20th', 'century', '.'], 'start': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'end': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [13]:
def get_word_embedding(word, voc, e_mat):
    if word in voc:
        return e_mat[voc[word], :]
    else:
        return e_mat[0, :]

def get_tokenize_sentences(documents):
    tokens = []
    
    for doc in documents:
        sents = nltk.sent_tokenize(doc)
        for sent in sents:
            sent = sent.strip(".")
            sent = re.sub(r'[,;":\']', '', sent)
            tokens.extend(nltk.word_tokenize(sent) )

    return tokens

def get_sent_embedding(sent, voc, emb_mat):
    sent_embedding = np.zeros((len(sent), 50))
    for i, word in enumerate(sent):
        word_embedding = get_word_embedding(word, voc, emb_mat)
        sent_embedding[i, :] = word_embedding

    sent_embedding = np.mean(sent_embedding, axis=0)
    return sent_embedding
    
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [98]:
test_ans = []

for t in testing_data[:3]:
    ans = dict()
    tokenize_sentences = get_tokenize_sentences(t['text'])
    tokenize_question = get_tokenize_sentences([t['question']])
    q_emb = get_sent_embedding(tokenize_question[0], voc, emb_mat)

    sims = np.zeros((len(tokenize_sentences)))
    for i, sent in enumerate(tokenize_sentences):
        s_emb = get_sent_embedding(sent, voc, emb_mat)
        sims[i] = cos_sim(q_emb, s_emb)
    
    print(sims)
    sentences = []
    for para in t['text']:
        sentences.extend(nltk.sent_tokenize(para))
    
    ans["id"] = t['id']
    ans['question'] = t['question']
    ans["text"] = sentences[np.argmax(sims)]
    test_ans.append(ans)

[0.93490492 0.90168545 0.92828754 0.91339167 0.86186674 0.82032573
 0.91350028 0.9100018 ]
[0.91581718 0.88284843 0.90909996 0.87392492 0.83246396 0.92219867
 0.91817481 0.90499963]
[0.9653651  0.94495417 0.98542792 0.97842912 0.94061607 0.9662019
 0.89187433 0.95549378 0.9673979  0.97011311 0.97152908 0.8997807 ]


In [99]:
file_name = "devel.json"
with open(file_name) as json_data:
    devel_set = json.load(json_data)

In [110]:
print(testing_data[2000])

{'question': 'What can adult learners obtain through studying at the Edwards Campus of the University of Kansas?', 'id': 2000, 'text': ['The University of Kansas School of Business is a public business school located on the main campus of the University of Kansas in Lawrence, Kansas.']}


In [19]:
def traverse(tree, tag="NP"):
    "recursively traverses an nltk.tree.Tree to find named entities"

    items = []

    if hasattr(tree, 'label') and tree.label:
        if tree.label() == tag:
            items.append(' '.join([child[0] for child in tree]))
        else:
            for child in tree:
                items.extend(traverse(child))

    return items

In [30]:
from nltk.tag import StanfordNERTagger
import nltk
import re


test_ans = []

grammar = r"""
  NUM:
    {<CD>}
  NP:
    {<DT>?<JJ>*<NN.*>+}
"""

snt = StanfordNERTagger('/home/ubuntu/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz') 
cp = nltk.RegexpParser(grammar)

for t in testing_data[:10]:
    answer = dict()
    answer["id"] = t['id']
    
    query = t["question"].lower()
    
    tokens = get_tokenize_sentences([t["text"]])
    ner_tag = snt.tag(tokens)
    
    persons = []
    for e in ner_tag:
        if e[1] == 'O':
            entities.append(e)
    print(entities)
    
    pos_tokens = nltk.pos_tag(tokens)
    tree = cp.parse(pos_tokens)
    
    if "who" in query:
        answer["text"]
        pass
    elif "when" in query:
        NPs = traverse(tree, "NUM")
        if not NPs:
            NPs = 
        pass
    elif "where" in query:
        pass
    else:
        
        pass
    
    test_ans.append(answer)
    

{'question': 'Modern browser support standards-based and defacto what?', 'id': 0, 'text': ' Modern web browsers support a combination of standards-based and de facto HTML and XHTML, which should be rendered in the same way by all browsers. In 1998, Netscape launched what was to become the Mozilla Foundation in an attempt to produce a competitive browser using the open source software model.'}
[('Mozilla', 'ORGANIZATION'), ('Foundation', 'ORGANIZATION')]
['Modern web browsers', 'a combination', 'HTML', 'XHTML', 'the same way', 'all browsers', '1998', 'Netscape', 'the Mozilla Foundation', 'an attempt', 'a competitive browser', 'the open source software model']
{'question': 'What do people typically call a web browser?', 'id': 1, 'text': ' A browser extension is a computer program that extends the functionality of a web browser. In 1998, Netscape launched what was to become the Mozilla Foundation in an attempt to produce a competitive browser using the open source software model.'}
[('Moz

In [1]:
import re

with open('test.csv', 'w') as f:
    f.write("id,answer\n")
    for t in testing_data:
        pos_sent = nltk.pos_tag(nltk.word_tokenize(t['text'][0]))
        tree = cp.parse(pos_sent)
        ans = traverse(tree)
        entities = snt.tag(traverse(tree))
        
        ans = " ".join(ans)
        ans = re.sub(r'[^\w\s]', '', ans)
        f.write(str(t['id']) + ',' + ans + '\n')

NameError: name 'testing_data' is not defined